In [1]:
import warnings
import pandas as pd
import pandasql as ps
import tableone as tbl1

warnings.filterwarnings('ignore')

In [2]:
df_sales = pd.read_csv('../data/sales_constant_prices_202502.csv')

In [3]:
df_sales.drop(['amount_euros_current_prices'], axis=1, inplace=True)

df_sales.rename(columns={'amount_euros_constant_prices_202502': 'amount_euros'}, inplace=True)

df_sales['transaction_date'] = pd.to_datetime(df_sales['transaction_date']).dt.date

df_sales['customer_id'] = df_sales['customer_id'].astype('str')

In [4]:
df_sales.head()

,customer_id,transaction_date,amount_euros,transaction_type
0,33829301,2016-06-10,119.449005,purchase
1,26742901,2015-01-17,17.580522,purchase
2,25929501,2015-04-06,106.607727,purchase
3,27012101,2015-04-10,30.120148,purchase
4,5362901,2015-05-17,239.902716,purchase


In [5]:
# Get summary statistics of df_sales
summary_statistics = df_sales.describe(include='all')

# Display the summary statistics
summary_statistics

,customer_id,transaction_date,amount_euros,transaction_type
count,634920,634920,634920.000000,634920
unique,115281,3734,NaN,2
top,27039601,2024-11-29,NaN,purchase
freq,319,671,NaN,611768
mean,NaN,NaN,162.240929,NaN
std,NaN,NaN,307.198807,NaN
min,NaN,NaN,-9795.324897,NaN
25%,NaN,NaN,23.702376,NaN
50%,NaN,NaN,56.990384,NaN
75%,NaN,NaN,184.496918,NaN


In [6]:
columns = ['amount_euros', 'transaction_type']

categorical = ['transaction_type']

# Create a TableOne object
summary_table = tbl1.TableOne(data=df_sales, columns=columns, categorical=categorical)

# Display the summary table
print(summary_table)

                                 Missing        Overall
n                                                634920
amount_euros, mean (SD)                0  162.2 (307.2)
transaction_type, n (%) purchase       0  611768 (96.4)
                        return              23152 (3.6)



In [7]:
date_sequence = pd.date_range(start='2021-01-01', end='2023-01-01', freq='MS')

df_analysis_dates = pd.DataFrame({'analysis_date':date_sequence})

df_analysis_dates['analysis_date_minus_1_yr'] = df_analysis_dates['analysis_date'] - pd.DateOffset(years=1)

df_analysis_dates['analysis_date_plus_1_yr'] = df_analysis_dates['analysis_date'] + pd.DateOffset(years=1)

df_analysis_dates = df_analysis_dates.apply(lambda col: col.dt.date)

In [8]:
query = '''
SELECT
    customer_id,
    analysis_date,
    MIN(transaction_date) AS first_purchase_date,
    MAX(transaction_date) AS last_purchase_date,
    SUM(amount_euros) AS total_purchase_amount
FROM df_sales AS s
INNER JOIN df_analysis_dates AS d
ON s.transaction_date < d.analysis_date
WHERE transaction_type = 'purchase'
GROUP BY customer_id, analysis_date
'''

df_features1 = ps.sqldf(query, locals())

df_features1['analysis_date'] = pd.to_datetime(df_features1['analysis_date']).dt.date
df_features1['first_purchase_date'] = pd.to_datetime(df_features1['first_purchase_date']).dt.date
df_features1['last_purchase_date'] = pd.to_datetime(df_features1['last_purchase_date']).dt.date


df_features1['recency'] = df_features1['analysis_date'] - df_features1['last_purchase_date']
df_features1['recency'] = df_features1['recency'].dt.days

df_features1['tenure'] = df_features1['analysis_date'] - df_features1['first_purchase_date']
df_features1['tenure'] = df_features1['tenure'].dt.days

df_features1.drop(['first_purchase_date', 'last_purchase_date'], axis=1, inplace=True)

In [9]:
query = '''
SELECT
    customer_id,
    analysis_date,
    COUNT(DISTINCT transaction_date) AS purchase_frequency_1_yr,
    SUM(amount_euros) AS purchase_amount_1_yr
FROM df_sales AS s
INNER JOIN df_analysis_dates AS d
ON s.transaction_date >= d.analysis_date_minus_1_yr AND s.transaction_date < d.analysis_date
WHERE transaction_type = 'purchase'
GROUP BY customer_id, analysis_date
'''

df_features2 = ps.sqldf(query, locals())

df_features2['analysis_date'] = pd.to_datetime(df_features2['analysis_date']).dt.date

In [12]:
query = '''
SELECT
    customer_id,
    analysis_date,
    COUNT(DISTINCT transaction_date) AS returns_frequency_1_yr,
    SUM(amount_euros) AS returned_amount_1_yr
FROM df_sales AS s
INNER JOIN df_analysis_dates AS d
ON s.transaction_date >= d.analysis_date_minus_1_yr AND s.transaction_date < d.analysis_date
WHERE transaction_type = 'return'
GROUP BY customer_id, analysis_date
'''

df_features3 = ps.sqldf(query, locals())

df_features3['analysis_date'] = pd.to_datetime(df_features3['analysis_date']).dt.date

In [10]:
query = '''
SELECT DISTINCT
    d.analysis_date,
    customer_id,
    0 AS is_churn
FROM df_sales AS s
INNER JOIN df_analysis_dates AS d
ON s.transaction_date >= d.analysis_date AND s.transaction_date < d.analysis_date_plus_1_yr
'''

df_labels = ps.sqldf(query, locals())

df_labels['analysis_date'] = pd.to_datetime(df_labels['analysis_date']).dt.date

In [14]:
query = '''
    SELECT 
        f1.customer_id,
        f1.analysis_date,
        f1.recency,
        f2.purchase_frequency_1_yr,
        f2.purchase_amount_1_yr,
        f1.tenure,
        f1.total_purchase_amount,
        COALESCE(f3.returns_frequency_1_yr,0) AS returns_frequency_1_yr,
        COALESCE(f3.returned_amount_1_yr,0) AS returned_amount_1_yr,
        COALESCE(c.is_churn, 1) AS is_churn
    FROM df_features1 AS f1
    INNER JOIN df_features2 AS f2
    ON f1.customer_id = f2.customer_id AND f1.analysis_date = f2.analysis_date
    LEFT JOIN df_features3 AS f3
    ON f1.customer_id = f3.customer_id AND f1.analysis_date = f3.analysis_date
    LEFT JOIN df_labels AS c
    ON f1.customer_id = c.customer_id AND f1.analysis_date = c.analysis_date
    '''

df_churn_dataset = ps.sqldf(query, locals())

df_churn_dataset.head()

,customer_id,analysis_date,recency,purchase_frequency_1_yr,purchase_amount_1_yr,tenure,total_purchase_amount,returns_frequency_1_yr,returned_amount_1_yr,is_churn
0,1,2021-01-01,253,1,349.552260,2189,1616.498520,0,0.0,0
1,1,2021-02-01,284,1,349.552260,2220,1616.498520,0,0.0,0
2,1,2021-03-01,312,1,349.552260,2248,1616.498520,0,0.0,0
3,1,2021-04-01,343,1,349.552260,2279,1616.498520,0,0.0,0
4,1,2021-09-01,3,1,464.269813,2432,2080.768333,0,0.0,0


In [15]:
df_churn_dataset.to_csv('../data/churn_dataset.csv', index=False)